## Taller de Programación en Python
### Profesor: Lucas Gómez Tobón

## Clase 7. Pandas avanzado

### Unir bases de datos (`merge`)

En la sesión anterior aprendimos como concatenar filas y columnas de diferentes bases de datos. Para hacer esto es necesario que la cantidad de columnas y filas, respectivamente, de los dataframes a juntar sean los mismos y que sus índices o llaves también lo sean.

No obstante, muchas veces cuando trate de juntar bases de datos, notará que no necesariamente todas las llaves están presentes en ambas bases de datos, o que incluso, a cada fila de la base izquierda, querrá pegarle más de una fila de la base derecha, o viceversa.

A la hora de hacer pegues más complejos, hablamos de que vamos a utilizar un `merge`. 

Comencemos con la sintaxis del `merge`. Para pegar dos bases de datos, usted usará un comando similar al siguiente:

```python
pd.merge(left = left_dataframe, right = right_dataframe, on = "alguna(s)_columa(s)", how = "left|right|inner|outer")`
```

Los argumentos que toma la función son:
- `left`: dataframe que va de primero.
- `right`: dataframe que va de segundo.
- `on`: es la columna o la lista de columnas que determinan qué filas de una tabla coinciden con qué filas de la segunda tabla. Comúnmente a estas variables se les llaman las llaves del pegue y debe identificar a cada observación de forma única. A veces, las columnas que desea fusionar tienen nombres diferentes en los datos. Por ejemplo, suponga que tiene dos bases de datos, una que registra el dinero mensual gastado por persona en almacenes Éxito y otra que tiene características personales de las personas. Usted podría tratar de juntar ambas bases con el identificador de fila o persona de cada base que en este caso podría ser la cédula, sin embargo, en un dataframe tal vez la variable se llame "cc" mientras que en el otro puede que se llame "cédula". En esos casos, puede especificar los nombres de columna por separado para cada marco de datos utilizando los argumentos "left_on" y "right_on".
- `how`: es el método a usar, por defecto Pandas usa el método "inner". Más adelante exploraremos más al respecto.

<center>
<div>
<img src="img/merges.png" width="400"/>
</div>
</center>

Tenemos cuatro grandes métodos para relacionar las bases porque no siempre tenemos una coincidencia uno a uno (one to one) entre las filas. Estos cuatro métodos afectan la forma en que Pandas trata los datos no coincidentes y eso es lo que veremos más adelante.

<center>
<div>
<img src="img/one-many.png" width="400"/>
</div>
</center>



In [2]:
import pandas as pd
import numpy as np

# ejemplos de pegues
left_dataframe = pd.DataFrame({"ID": [1,2,3,4], "left_side": "Izquierda"})
right_dataframe = pd.DataFrame({"ID": [3,4,5,6], "right_side": "Derecha"})

In [3]:
left_dataframe

,ID,left_side
0,1,Izquierda
1,2,Izquierda
2,3,Izquierda
3,4,Izquierda


In [4]:
right_dataframe

,ID,right_side
0,3,Derecha
1,4,Derecha
2,5,Derecha
3,6,Derecha


#### Left merge
En un Left merge lo que más nos interesa son los datos del lado IZQUIERDO a los cuales queremos pegarles columnas de una base de datos en el lado DERECHO.

Para hacer eso, cortamos las filas en el marco de datos DERECHO y pegamos partes en el marco de datos IZQUIERDO. Recuerde, nos preocupamos principalmente por el lado IZQUIERDO y solo queremos datos del lado DERECHO si tiene alguna de las mismas ID. Entonces, si algo en el marco de datos DERECHO no coincide o no existe, entonces tenemos que hacer cosas para mantener las columnas de la misma longitud. Lo hacemos agregando NaN para llenar el vacío o descartando algunas filas por completo.

En este ejemplo, el lado IZQUIERDO tiene los ID 1, 2, 3 y 4:
- El lado DERECHO no tiene ID 1 o 2, por lo que agregamos NaN porque necesitamos que las columnas tengan la misma longitud.
- El lado DERECHO tiene datos para los ID 3 y 4, así que lo agregamos como una nueva columna.
- El lado IZQUIERDO no tiene ID 5 o 6, por lo que no necesitamos esa información del DERECHO y se descarta.

<center>
<div>
<img src="img/left_merge.png" width="400"/>
</div>
</center>

In [5]:
# Left merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "left")

,ID,left_side,right_side
0,1,Izquierda,NaN
1,2,Izquierda,NaN
2,3,Izquierda,Derecha
3,4,Izquierda,Derecha


#### Right merge
Los Right merges funcionan igual que los Left merges, la diferencia es que nos preocupamos principalmente por el lado DERECHO y nos gustaría agregar datos desde el IZQUIERDO si tienen ID coincidentes.

<center>
<div>
<img src="img/right_merge.png" width="400"/>
</div>
</center>

In [6]:
# Right merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "right")

,ID,left_side,right_side
0,3,Izquierda,Derecha
1,4,Izquierda,Derecha
2,5,NaN,Derecha
3,6,NaN,Derecha


#### Inner merge
Con un Inner merge, cortamos ambos marcos de datos y solo pegamos las cosas que coinciden. Si una ID no está en ambos marcos de datos, no la mantenemos y no agregamos NaN.

<center>
<img src="img/right_merge.png" width="400"/>
</center>

In [7]:
# Inner merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "inner")

,ID,left_side,right_side
0,3,Izquierda,Derecha
1,4,Izquierda,Derecha


#### Outer merge
Con un Outer merge, cortamos ambos marcos de datos y mantenemos todo de ambos lados. Luego agregamos NaN para llenar los espacios en blanco.

<center>
<img src="img/outer_merge.png" width="400"/>
</center>

In [8]:
# Outer merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "outer")

,ID,left_side,right_side
0,1,Izquierda,NaN
1,2,Izquierda,NaN
2,3,Izquierda,Derecha
3,4,Izquierda,Derecha
4,5,NaN,Derecha
5,6,NaN,Derecha


Ahora haremos un ejemplo más práctico. Importaremos dos bases de datos sobre usuarios que califican restaurantes en internet. La primera llamada `payment` informa el método de pago favorito para cada cliente y la segunda, `profile` describe algunas características sociodemográficas de los clientes. En este caso nuestra idea es juntar ambas bases de datos.

In [9]:
# Importe bases de datos
payment = pd.read_csv("../Datos/userpayment.csv")
profile = pd.read_csv("../Datos/userprofile.csv")

In [10]:
# Analicemos la estructura de las bases
payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userID    177 non-null    object
 1   Upayment  177 non-null    object
dtypes: object(2)
memory usage: 2.9+ KB


In [11]:
payment.head()

,userID,Upayment
0,U1001,cash
1,U1002,cash
2,U1003,cash
3,U1004,cash
4,U1004,bank_debit_cards


In [12]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   userID            138 non-null    object 
 1   latitude          138 non-null    float64
 2   longitude         138 non-null    float64
 3   smoker            138 non-null    object 
 4   drink_level       138 non-null    object 
 5   dress_preference  138 non-null    object 
 6   ambience          138 non-null    object 
 7   transport         138 non-null    object 
 8   marital_status    138 non-null    object 
 9   hijos             138 non-null    object 
 10  birth_year        138 non-null    int64  
 11  interest          138 non-null    object 
 12  personality       138 non-null    object 
 13  religion          138 non-null    object 
 14  activity          138 non-null    object 
 15  color             138 non-null    object 
 16  weight            138 non-null    int64  
 1

In [13]:
profile.head()

,userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,personality,religion,activity,color,weight,budget,height
0,U1001,22.139997,-100.978803,false,abstemious,informal,family,on foot,single,independent,1989,variety,thrifty-protector,none,student,black,69,medium,1.77
1,U1002,22.150087,-100.983325,false,abstemious,informal,family,public,single,independent,1990,technology,hunter-ostentatious,Catholic,student,red,40,low,1.87
2,U1003,22.119847,-100.946527,false,social drinker,formal,family,public,single,independent,1989,none,hard-worker,Catholic,student,blue,60,low,1.69
3,U1004,18.867000,-99.183000,false,abstemious,informal,family,public,single,independent,1940,variety,hard-worker,none,professional,green,44,medium,1.53
4,U1005,22.183477,-100.959891,false,abstemious,no preference,family,public,single,independent,1992,none,thrifty-protector,Catholic,student,black,65,medium,1.69


Note que ambas bases tienen diferente número de observaciones, `profile` tiene 133 observaciones mientras que `payment` tiene 177 clientes. Esto quiere decir que, aunque profile es una caracterización más completa de los clientes, payment tiene más observaciones. Adicionalmente, ninguna de las bases tiene NAs.

Al parecer `userID` corresponde a la llave/identificador de cada cliente/fila. Revisemos que no hayan duplicados!

In [14]:
payment.userID.duplicated().sum()

44

In [15]:
profile.userID.duplicated().sum()

0

Mientras que `profile` no tiene duplicados, `payment` tiene 44 duplicados, vamos a revisarlos. Al parecer ambas bases tienen los mismos clientes, lo que pasa es que algunos tienen más de un tipo de método de pago.

In [16]:
# Devolver todos los duplicados
payment.loc[payment.userID.duplicated(False),]

,userID,Upayment
3,U1004,cash
4,U1004,bank_debit_cards
12,U1012,cash
13,U1012,bank_debit_cards
14,U1013,MasterCard-Eurocard
...,...,...
155,U1117,cash
159,U1121,cash
160,U1121,bank_debit_cards
170,U1133,bank_debit_cards


In [17]:
# Vamos a eliminar los duplicados dejando solo la primera observación. Suponemos que el primer método de pago es el 
# más deseado
payment = payment.drop_duplicates(subset = ["userID"], keep = "first").reset_index(drop = True)

Como la base que más nos interesa es la de `profile` vamos a hacer que esta sea nuestra base de la IZQUIERDA y hacer un LEFT merge

In [18]:
df = pd.merge(left = profile, right = payment, on = "userID", how = "left")
df.head()

,userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,personality,religion,activity,color,weight,budget,height,Upayment
0,U1001,22.139997,-100.978803,false,abstemious,informal,family,on foot,single,independent,1989,variety,thrifty-protector,none,student,black,69,medium,1.77,cash
1,U1002,22.150087,-100.983325,false,abstemious,informal,family,public,single,independent,1990,technology,hunter-ostentatious,Catholic,student,red,40,low,1.87,cash
2,U1003,22.119847,-100.946527,false,social drinker,formal,family,public,single,independent,1989,none,hard-worker,Catholic,student,blue,60,low,1.69,cash
3,U1004,18.867000,-99.183000,false,abstemious,informal,family,public,single,independent,1940,variety,hard-worker,none,professional,green,44,medium,1.53,cash
4,U1005,22.183477,-100.959891,false,abstemious,no preference,family,public,single,independent,1992,none,thrifty-protector,Catholic,student,black,65,medium,1.69,cash


Debemos revisar que todos los elementos en profile hayan encontrado un match exacto en payment

In [19]:
df["Upayment"].isna().sum()

5

Upa! Tenemos 5 NAs. Eso quiere decir que hay 5 usuarios/clientes en profile que no están en payment! Revisemos

In [20]:
usuarios_faltantes = df.loc[df["Upayment"].isna(), "userID"].values
usuarios_faltantes

array(['U1024', 'U1025', 'U1088', 'U1122', 'U1130'], dtype=object)

In [21]:
payment["userID"].isin(usuarios_faltantes).sum()

0

In [22]:
# En efecto, estos 5 usuarios no están en la base de payment
set(profile["userID"]) - set(payment["userID"])

{'U1024', 'U1025', 'U1088', 'U1122', 'U1130'}

In [23]:
# Sin embargo, en la base de profile sí están todos los usuarios de payment
set(payment["userID"]) - set(profile["userID"])

set()

In [24]:
profile.shape

(138, 19)

In [25]:
payment.shape

(133, 2)

### Groupby


Uno de los métodos más útiles para los analistas de datos es `.groupby()`. Este método permite dividir los datos en grupos y a cada uno de estos aplicarles una función de agregación.

Veamos el siguiente ejemplo para entender este concepto mejor:

In [26]:
df = pd.read_excel("../Datos/ejemplo_groupby.xlsx")
df

,animal,age,weight,length
0,hamster,1,7,8
1,alligator,9,13,6
2,hamster,4,8,9
3,cat,13,12,1
4,snake,14,11,8
5,cat,10,8,9
6,hamster,2,10,5
7,cat,4,14,6
8,cat,14,9,6
9,snake,7,11,6


Note que tenemos un `dataframe` con cuatro tipos de animales: 
- alligators (cocodrilos 🐊)
- cats (gatos 🐱)
- snakes (serpientes 🐍)
- hamsters (hamsters 🐹)

Cada una de las filas indican un chequeo en el veterinario donde se registra edad, peso y largo del animal. Por ende, usted como investigador quiere estudiar algunas estadísticas descriptivas por especie. Por ejemplo ¿Cuál es el peso promedio de cada especie?

In [27]:
# El primer paso es agrupar por animal
animal_groups = df.groupby("animal")

In [28]:
animal_groups

In [29]:
# Veamos la conformación de cada uno de los grupos. ¿En qué filas aparece cada animal?
animal_groups.groups

{'alligator': [1, 13], 'cat': [3, 5, 7, 8, 12], 'hamster': [0, 2, 6, 10, 11], 'snake': [4, 9]}

In [30]:
# El segundo paso es aplicar una funcion agregadora
# ¿Cuál es la media del peso por especie?
animal_groups["weight"].mean()

animal
alligator    13.5
cat          10.4
hamster       9.0
snake        11.0
Name: weight, dtype: float64

Visualmente, lo que sucedió fue lo siguiente:

1. Se agrupa los valores únicos de la columna animal.
<center>
<img src = "img/groupby1.jpg" width = "400">
</center>

2. La segmentación de cada grupo se vería de la siguiente manera
<center>
<img src = "img/groupby2.jpg" width = "400">
</center>

3. Se le asignan las otras variables/columnas a cada grupo
<center>
<img src = "img/groupby3.jpg" width = "400">
</center>

4. Se aplica la función agregadora `.mean()` sobre la columna `weight` de cada grupo.
<center>
<img src = "img/groupby4.jpg" width = "400">
</center>


In [31]:
# Probemos otros ejemplos
# ¿Cuál es la edad mediana por animal?
df.groupby("animal")["age"].median()

animal
alligator     8.0
cat          10.0
hamster       2.0
snake        10.5
Name: age, dtype: float64

In [32]:
# ¿Cuál es el largo máximo por animal?
df.groupby("animal")["length"].max()

animal
alligator    6
cat          9
hamster      9
snake        8
Name: length, dtype: int64

In [33]:
# ¿Cuál es la desviación estándar del peso por animal?
df.groupby("animal")["weight"].std()

animal
alligator    0.707107
cat          2.509980
hamster      1.414214
snake        0.000000
Name: weight, dtype: float64

Para seguir practicando, vamos a utilizar los dos métodos vistos en clase (`.merge()` y `.groupby()`) en un solo ejercicio. Vamos a combinar una base de datos que contiene restaurantes y sus respectivas calificaciones con otra que contiene la información del tipo de parqueadero que tiene cada restaurante: `[None, Public, Valet, Yes]`.

La pregunta que queremos resolver con este ejercicio es cómo el parqueadero puede influir en la calificación o percepción de los clientes sobre un restaurante.

In [34]:
# 1. Importe los datos
ratings = pd.read_csv("../Datos/rating_final.csv")
parking = pd.read_csv("../Datos/chefmozparking.csv")

In [35]:
# Inspeccione los datos
ratings.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [36]:
# Se puede ver que userID se refiere al identificador de usuario que calificó al restaurante placeID. 
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userID          1161 non-null   object
 1   placeID         1161 non-null   int64 
 2   rating          1161 non-null   int64 
 3   food_rating     1161 non-null   int64 
 4   service_rating  1161 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 45.5+ KB


In [37]:
# Inspeccionemos la base de parking
parking.head()

,placeID,parking_lot
0,135111,public
1,135110,none
2,135109,none
3,135108,none
4,135107,none


In [39]:
# Para cada restaurante (placeID) se tiene una descripción del tipo de parqueadero.
# Estudiemos cuántos tipos de parqueaderos tiene cada restaurante
parking.placeID.value_counts().describe()

count    675.00000
mean       1.04000
std        0.20353
min        1.00000
25%        1.00000
50%        1.00000
75%        1.00000
max        3.00000
Name: placeID, dtype: float64

In [54]:
# Veamos la proporción de tipos de parqueaderos
parking.parking_lot.value_counts(normalize = True)

none                 0.495726
yes                  0.247863
public               0.145299
street               0.045584
fee                  0.031339
valet parking        0.029915
validated parking    0.004274
Name: parking_lot, dtype: float64

In [ ]:
# En general cada restaurante tiene un sólo tipo de parqueadero pero hay algunos que tienen más de 1 tipo
# Pregunta: ¿Cuál es la variable con la que queremos hacer el pegue?
# ¿Qué tipo de pegue queremos hacer?

In [43]:
# Queremos hacer el pegue con la variable placeID.
# Debemos verificar que ambas variables estén en el mismo formato
ratings.placeID.dtype

dtype('int64')

In [42]:
parking.placeID.dtype

dtype('int64')

In [44]:
# Esto debe ser True siempre
ratings.placeID.dtype == parking.placeID.dtype

True

In [46]:
# Vamos a hacer un left join porque queremos tener absolutamente todas las calificaciones de los restaurantes
ratings = ratings.merge(parking, on = "placeID", how = "left")

In [49]:
# Veamos que tan bueno estuvo el pegue

# ¿Cuál es la cantidad de NAs o valores faltantes por variable?
ratings.isna().sum()

userID            0
placeID           0
rating            0
food_rating       0
service_rating    0
parking_lot       0
dtype: int64

In [50]:
# ¿Cuál es la proporción de NAs o valores faltantes por variable?
ratings.isna().mean()

userID            0.0
placeID           0.0
rating            0.0
food_rating       0.0
service_rating    0.0
parking_lot       0.0
dtype: float64

In [51]:
# Note que hay 0 NAs en parking_lot sin embargo hay algunos parqueaderos con None. Note que un NA no es lo mismo a None
np.nan == None

False

**¿Como hacemos para analizar las variables de rating a la luz del tipo de parqueo?**

In [61]:
ratings.groupby("parking_lot")[["rating", "food_rating", "service_rating"]].mean() \
    .round(2).sort_values("service_rating", ascending = False)

,rating,food_rating,service_rating
parking_lot,,,
valet parking,1.34,1.34,1.34
none,1.20,1.21,1.10
yes,1.21,1.21,1.09
public,1.15,1.22,1.02


¿Qué pasaría si no quisiera tener solo la media sino otras estadísticas más completas?

#### Método .agg()
El método .agg() se puede utilizar después de aplicar un método .groupby() en pandas para realizar operaciones de agregación en los datos de cada grupo.

La sintaxis general de la función .groupby() es la siguiente:
```python
dataframe.groupby(columnas).agg(funciones)
```
Donde:
- dataframe: el DataFrame al que se aplicará la función `groupby()`.
- columnas: la(s) columna(s) que se utilizarán para agrupar los datos.
- funciones: la(s) operación(es) de agregación que se aplicarán a los datos agrupados.

Por ejemplo, para calcular la media, el máximo y el mínimo de las columnas de rating del DataFrame agrupado por la columna 'parking_lot', se puede utilizar la siguiente sintaxis:

In [62]:
ratings.groupby("parking_lot")[["rating", "food_rating", "service_rating"]].agg(["min", "mean", "max"])

rating               food_rating               service_rating  \
                 min      mean max         min      mean max            min   
parking_lot                                                                   
none               0  1.203209   2           0  1.212121   2              0   
public             0  1.148352   2           0  1.219780   2              0   
valet parking      0  1.344828   2           0  1.344828   2              0   
yes                0  1.208226   2           0  1.208226   2              0   

                             
                   mean max  
parking_lot                  
none           1.098039   2  
public         1.021978   2  
valet parking  1.344828   2  
yes            1.092545   2

También es posible utilizar varias columnas para agrupar los datos y aplicar diferentes operaciones de agregación a diferentes columnas. Por ejemplo:

In [71]:
ratings.groupby("parking_lot").agg({'rating': ['mean', 'max'], 'food_rating': 'std', 
                                     "service_rating": lambda x: np.percentile(x, 50)})

rating     food_rating service_rating
                   mean max         std       <lambda>
parking_lot                                           
none           1.203209   2    0.783488            1.0
public         1.148352   2    0.804713            1.0
valet parking  1.344828   2    0.813979            2.0
yes            1.208226   2    0.799700            1.0

In [80]:
# Otra sintaxis, en vez de un diccionario, usar tuplas
ratings.groupby("parking_lot").agg(rating_media = ("rating", 'mean'), 
                                   rating_maximo = ("rating", 'max'),
                                   service_rating_mediana = ("service_rating", lambda x: np.percentile(x, 50)))

,rating_media,rating_maximo,service_rating_mediana
parking_lot,,,
none,1.203209,2,1.0
public,1.148352,2,1.0
valet parking,1.344828,2,2.0
yes,1.208226,2,1.0


In [82]:
ratings.groupby("parking_lot")[["rating", "food_rating", "service_rating"]].describe()

rating                                              food_rating  \
               count      mean       std  min  25%  50%  75%  max       count   
parking_lot                                                                     
none           561.0  1.203209  0.777857  0.0  1.0  1.0  2.0  2.0       561.0   
public         182.0  1.148352  0.768850  0.0  1.0  1.0  2.0  2.0       182.0   
valet parking   29.0  1.344828  0.768852  0.0  1.0  2.0  2.0  2.0        29.0   
yes            389.0  1.208226  0.770148  0.0  1.0  1.0  2.0  2.0       389.0   

                         ...           service_rating                      \
                   mean  ...  75%  max          count      mean       std   
parking_lot              ...                                                
none           1.212121  ...  2.0  2.0          561.0  1.098039  0.799115   
public         1.219780  ...  2.0  2.0          182.0  1.021978  0.764951   
valet parking  1.344828  ...  2.0  2.0           29.0  1.344828  0.813979   
yes            1.208226  ...  2.0  2.0          389.0  1.092545  0.787578   

                                        
               min  25%  50%  75%  max  
parking_lot                             
none           0.0  0.0  1.0  2.0  2.0  
public         0.0  0.0  1.0  2.0  2.0  
valet parking  0.0  1.0  2.0  2.0  2.0  
yes            0.0  0.0  1.0  2.0  2.0  

[4 rows x 24 columns]